<a href="https://colab.research.google.com/github/dlsyscourse/lecture14/blob/main/14_hardware_acceleration_architecture_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 14: Hardware Acceleration Implementation

In this lecture, we will to walk through backend scafoldings to get us hardware accelerations for needle.




## Select a GPU runtime type
In this lecture, we are going to make use of c++ and CUDA to build accelerated linear algebra libraries. In order to do so, please make sure you select a runtime type with GPU and rerun the cells if needed:
- Click on the "Runtime" tab
- Click "Change runtime type"
- Select GPU

After you started the right runtime, you can run the following command to check if there is a GPU available.

In [1]:
!nvidia-smi

Mon Nov 14 02:26:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:1A:00.0 Off |                  Off |
| N/A   27C    P0    24W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:1B:00.0 Off |                  Off |
| N/A   

## Prepare the codebase

To get started, we can clone the related repo from the github. 

In [ ]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
!mkdir -p 10714f22
%cd /content/drive/MyDrive/10714f22
# comment out the following line if you run it for the second time
# as you already have a local copy of lecture14
# !git clone https://github.com/dlsyscourse/lecture14 
!ln -s /content/drive/MyDrive/10714f22/lecture14 /content/needle

Mounted at /content/drive
/content/drive/MyDrive
/content/drive/MyDrive/10714f22
Cloning into 'lecture14'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 53 (delta 15), reused 50 (delta 12), pack-reused 0
Unpacking objects: 100% (53/53), done.


In [2]:
!python -m pip install pybind11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.1/216.1 kB 1.9 MB/s eta 0:00:0000:0100:01


### Build the needle cuda library

We leverage pybind to build a c++/cuda library for acceleration. You can type make to build the corresponding library.

In [1]:
# %cd /content/needle
!make clean
!make

rm -rf build python/needle/backend_ndarray/ndarray_backend*.so
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Python: /home/gehao/anaconda3/envs/lxy/bin/python (found version "3.7.0") found components: Development Interpreter Development.Module Development.Embed 
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- Found pybind11: /home/gehao/anaconda3/envs/lxy/lib/python3.7/site-packages/pybind11/include (found version "2.10.1")
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HA

We can then run the following command to make the path to the package available in colab's environment as well as the PYTHONPATH.

In [4]:
%set_env PYTHONPATH python:/env/python
import sys
sys.path.append("python")

env: PYTHONPATH=python:/env/python


## Codebase walkthrough


Now click the files panel on the left side. You should be able to see these files

Python:
- needle/backend_ndarray/ndarray.py
- needle/backend_ndarray/ndarray_backend_numpy.py

C++/CUDA
- src/ndarray_backend_cpu.cc
- src/ndarray_backend_cuda.cu

The main goal of this lecture is to create an accelerated ndarray library.
As a result, we do not need to deal with needle.Tensor for now and will focus on backend_ndarray's implementation. 

After we build up this array library, we can then use it to power backend array computations in needle.


## Creating a CUDA NDArray






In [5]:
from needle import backend_ndarray as nd

In [6]:
x = nd.NDArray([1, 2, 3], device=nd.cpu())

In [7]:
x = nd.NDArray([1,2,3], device=nd.cuda())

In [8]:
y = x + 1

In [9]:
y

NDArray([2. 3. 4.], device=cuda())

In [10]:
y = x + x

In [11]:
y

NDArray([2. 4. 6.], device=cuda())

We can create a CUDA tensor from the data by specifying a device keyword.

In [12]:
x = nd.NDArray([1, 2, 3], device=nd.cuda())

In [13]:
y = x + 1

In [14]:
x.numpy()

array([1., 2., 3.], dtype=float32)

In [15]:
x.device

cuda()

In [16]:
y = x + 1

In [17]:
y.device

cuda()

In [18]:
y.numpy()

array([2., 3., 4.], dtype=float32)

### Key Data Structures

Key data structures in backend_ndarray

- NDArray: the container to hold device specific ndarray
- BackendDevice: backend device
    - mod holds the module implementation that implements all functions
    - checkout ndarray_backend_numpy.py for a python-side reference.



## Trace GPU execution

Now, let us take a look at what happens when we execute the following code


In [19]:
x = nd.NDArray([1, 2, 3], device=nd.cuda())
y = x + 1

In [20]:
x.device.from_numpy

<function needle.backend_ndarray.ndarray_backend_cuda.PyCapsule.from_numpy>

In [21]:
x = nd.NDArray([1, 2, 3])

In [22]:
x.device.from_numpy

<function needle.backend_ndarray.ndarray_backend_numpy.from_numpy(a, out)>

Have the following trace:

backend_ndarray/ndarray.py
- `NDArray.__add__`
- `NDArray.ewise_or_scalar`
- `ndarray_backend_cpu.cc:ScalarAdd`

In [23]:
y.numpy()

array([2., 3., 4.], dtype=float32)

Have the following trace:

- `NDArray.numpy`
- `ndarray_backend_cpu.cc:to_numpy`

## Guidelines for Reading C++/CUDA related Files

Read
- src/ndarray_backend_cpu.cc
- src/ndarray_backend_cuda.cu


Optional
- CMakeLists.txt: this is used to setup the build and likely you do not need to tweak it.







## NDArray Data Structure

Open up `python/needle/backend_ndarray/ndarray.py`.

An NDArray contains the following fields:
- handle: The backend handle that build a flat array which stores the data.
- shape: The shape of the NDArray
- strides: The strides that shows how do we access multi-dimensional elements
- offset: The offset of the first element.
- device: The backend device that backs the computation






## Transformation as Strided Computation

We can leverage the strides and offset to perform transform/slicing with zero copy.

- Broadcast: insert strides that equals 0
- Tranpose: swap the strides
- Slice: change the offset and shape 

For most of the computations, however, we will call `array.compact()` first to get a contiguous and aligned memory before running the computation.

In [24]:
import numpy as np
x = nd.NDArray([0, 1,2,3,4,5], device=nd.cpu_numpy())

In [25]:
x.numpy()

array([0., 1., 2., 3., 4., 5.], dtype=float32)

In [26]:
y = nd.NDArray.make(shape=(3, 2, 2), strides=(2, 1, 0), device=x.device, handle=x._handle, offset=0)
y.numpy()

array([[[0., 0.],
        [1., 1.]],

       [[2., 2.],
        [3., 3.]],

       [[4., 4.],
        [5., 5.]]], dtype=float32)

In [27]:
x = nd.NDArray([1, 2, 3, 4], device=nd.cpu_numpy())

In [28]:
x.numpy()

array([1., 2., 3., 4.], dtype=float32)

We can use strides and shape manipulation to create different views of the same array.

In [29]:
y = nd.NDArray.make(shape=(2, 2), strides=(2, 1), device=x.device, handle=x._handle, offset=0)

In [30]:
y.numpy()

array([[1., 2.],
       [3., 4.]], dtype=float32)

In [31]:
z = nd.NDArray.make(shape=(2, 1), strides=(2, 1), device=x.device, handle=x._handle, offset=1)

In [32]:
z.numpy()

array([[2.],
       [4.]], dtype=float32)

## CUDA Acceleration

Now let us open `src/ndarray_cuda_backend.cu` and take a look at current implementation of GPU ops.


## Steps for adding a new operator implementation
- Add an implementation in `ndarray_backend_cuda.cu`, expose via pybind
- Call into the operator in ndarray.py
- Write up testcases

In [41]:
!make

-- Found pybind11: /home/gehao/anaconda3/envs/lxy/lib/python3.7/site-packages/pybind11/include (found version "2.10.1")
-- Found cuda, building cuda backend
Mon Nov 14 07:11:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:1A:00.0 Off |                  Off |
| N/A   29C    P0    31W / 250W |    257MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+----------

In [34]:
x = nd.NDArray([1,2,3], device=nd.cuda())

In [43]:
!python tests/test_mul.py

[3. 6. 9.]


## Connect back to needle Tensor

So far we only played with the `backend_ndarray` subpackage, which is a self-contained ndarray implementation within needle.

We can connect the ndarray back to needle as a backend.

In [36]:
import needle as ndl

In [37]:
x = ndl.Tensor([1,2,3], device=ndl.cpu(), dtype="float32")
y = ndl.Tensor([2,3,5], device=ndl.cpu(), dtype="float32")
z = x + y

In [38]:
z.device

cpu()

In [39]:
type(z.cached_data)

needle.backend_ndarray.ndarray.NDArray

## Write Standalone Python Test Files

Now that we have additional c++/cuda libraries in needle, we will need to type make in order to rebuild the library. Additionally, because the colab environment caches the old library, it is inconvenient to use the ipython cells to debug the updated library.




In [ ]:
!make

-- Found pybind11: /usr/local/lib/python3.7/dist-packages/pybind11/include (found version "2.8.1" )
-- Found cuda, building cuda backend
-- Autodetected CUDA architecture(s):  3.7
-- Configuring done
-- Generating done
-- Build files have been written to: /content/drive/MyDrive/10714/lecture17/build
make[1]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
make[2]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
make[3]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
make[3]: Leaving directory '/content/drive/MyDrive/10714/lecture17/build'
[  0%] Built target ndarray_backend_cuda
make[3]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
make[3]: Leaving directory '/content/drive/MyDrive/10714/lecture17/build'
[ 50%] Built target ndarray_backend_cpu
make[2]: Leaving directory '/content/drive/MyDrive/10714/lecture17/build'
make[1]: Leaving directory '/content/drive/MyDrive/10714/lecture17/build'



We recommend writing separate python files and invoke them from the command line. Create a new file `tests/mytest.py` and write your local tests. This is also a common develop practice in big projects that involves python c++ FFI.

In [ ]:
!python tests/mytest.py

python3: can't open file 'tests/mytest.py': [Errno 2] No such file or directory


After we have building the library, we could choose to fully restart the runtime (factory reset runtime) if you want to bring the updated change back to another colab. Note that you will need to save your code changes to the drive or a private github repo.